**<파일 경로>**
- C:/Users/Brian/Desktop/Data Science/Projects/Playdata/dataset/made_by_전처리/test

In [1]:
filepath = input()

C:/Users/Brian/Desktop/Data Science/Projects/Playdata/dataset/made_by_전처리/test


In [2]:
import numpy as np
import pandas as pd
import datetime as dt
from tqdm.notebook import tqdm

# 최종 데이터 셋 생성
- 앞서 전처리 작업을 거친 "지하철 데이터", "인구 데이터", "기상 데이터"들을 결합하여 최종 데이터 셋을 만들도록 하겠다.

- 먼저 메모리 절약을 위해 각 변수들의 데이터 타입을 지정해 준 뒤, 각 데이터 셋을 불러오도록 하겠다.

In [3]:
# 지하철 데이터 셋의 변수들 데이터 타입 지정
subway_dtype = {
    'hour': np.int32, 'boarding_num': np.int32, 'waiting_area': np.float32, 'platform_area': np.float32, 
    'length': np.int32, 'transfer_num': np.int32, 'entrance': np.int32, 'msi_cnt': np.int32, 
    'hsi_cnt': np.int32, 'uvi_cnt': np.int32, 'hpi_cnt': np.int32, 'mti_cnt': np.int32
}

# 인구 데이터 셋의 변수들 데이터 타입 지정
pop_dtype = {
    'hour': np.int32, 'total': np.float32, 'pop_below_20': np.float32,
    'pop_20': np.float32, 'pop_30': np.float32, 'pop_40': np.float32, 
    'pop_50': np.float32, 'pop_60': np.float32, 'pop_over_70': np.float32
}

# 기상 데이터 셋의 변수들 데이터 타입 지정
weather_dtype = {
    'temp': np.float32, 'windspeed': np.float32, 'rain': np.float32, 'hour': np.int32
}

In [4]:
subway_final = pd.read_csv('{}/subway_final.csv'.format(filepath), dtype = subway_dtype)
pop_final = pd.read_csv('{}/pop_final.csv'.format(filepath), dtype = pop_dtype)
weather_final = pd.read_csv('{}/weather_final.csv'.format(filepath), dtype = weather_dtype)

C:\Users\Brian\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


- 추가적인 메모리 절약을 위해 object 타입 변수들을 모두 category 타입으로 변환해주겠다.

In [5]:
subway_cat = ['date', 'line', 'station_name', 'boarding', 'type', 'floor', 
              'transfer', 'gate', 'gu', 'dong', 'pop_join_key', 'weather_join_key']

pop_cat = ['date', 'pop_join_key']

weather_cat = ['date', 'weather_join_key']

for cat in tqdm(subway_cat):
    subway_final[cat] = subway_final[cat].astype('category')
    
for cat in tqdm(pop_cat):
    pop_final[cat] = pop_final[cat].astype('category')
    
for cat in tqdm(weather_cat):
    weather_final[cat] = weather_final[cat].astype('category')

In [6]:
subway_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1867120 entries, 0 to 1867119
Data columns (total 24 columns):
 #   Column            Dtype   
---  ------            -----   
 0   date              category
 1   line              category
 2   station_name      category
 3   boarding          category
 4   hour              int32   
 5   boarding_num      int32   
 6   waiting_area      float32 
 7   platform_area     float32 
 8   gu                category
 9   type              category
 10  length            int32   
 11  floor             category
 12  transfer          category
 13  transfer_num      int32   
 14  entrance          int32   
 15  gate              category
 16  msi_cnt           int32   
 17  hsi_cnt           int32   
 18  uvi_cnt           int32   
 19  hpi_cnt           int32   
 20  mti_cnt           int32   
 21  dong              category
 22  pop_join_key      category
 23  weather_join_key  category
dtypes: category(12), float32(2), int32(10)
memory usag

In [7]:
pop_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1847664 entries, 0 to 1847663
Data columns (total 11 columns):
 #   Column        Dtype   
---  ------        -----   
 0   date          category
 1   hour          int32   
 2   total         float32 
 3   pop_below_20  float32 
 4   pop_20        float32 
 5   pop_30        float32 
 6   pop_40        float32 
 7   pop_50        float32 
 8   pop_60        float32 
 9   pop_over_70   float32 
 10  pop_join_key  category
dtypes: category(2), float32(8), int32(1)
memory usage: 77.1 MB


In [8]:
weather_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109200 entries, 0 to 109199
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype   
---  ------            --------------   -----   
 0   date              109200 non-null  category
 1   temp              109200 non-null  float32 
 2   windspeed         109200 non-null  float32 
 3   rain              109200 non-null  float32 
 4   hour              109200 non-null  int32   
 5   weather_join_key  109200 non-null  category
dtypes: category(2), float32(3), int32(1)
memory usage: 2.3 MB


# 3개의 데이터 셋(지하철+인구+기상) 결합
- 앞서 만들어놓은 조인키를 사용하여 3개의 데이터 셋(지하철 + 인구 + 기상)을 결합해주겠다.

In [9]:
subway_final.shape, weather_final.shape, pop_final.shape

((1867120, 24), (109200, 6), (1847664, 11))

In [10]:
# subway_final + weather_final
subway_weather_final = pd.merge(subway_final, weather_final, how = 'left', on = ['weather_join_key', 'date', 'hour'])
subway_weather_final

,date,line,station_name,boarding,hour,boarding_num,waiting_area,platform_area,gu,type,...,hsi_cnt,uvi_cnt,hpi_cnt,mti_cnt,dong,pop_join_key,weather_join_key,temp,windspeed,rain
0,2020-01-01,1호선,서울역,승차,0,22,8725.0,2080.000000,중구,섬식,...,1,0,0,0,회현동,2020-01-01_회현동,2020-01-01_중구,-7.100000,0.7,0.0
1,2020-01-01,1호선,서울역,하차,0,74,8725.0,2080.000000,중구,섬식,...,1,0,0,0,회현동,2020-01-01_회현동,2020-01-01_중구,-7.100000,0.7,0.0
2,2020-01-01,1호선,서울역,승차,5,316,8725.0,2080.000000,중구,섬식,...,1,0,0,0,회현동,2020-01-01_회현동,2020-01-01_중구,-6.800000,0.4,0.0
3,2020-01-01,1호선,서울역,하차,5,234,8725.0,2080.000000,중구,섬식,...,1,0,0,0,회현동,2020-01-01_회현동,2020-01-01_중구,-6.800000,0.4,0.0
4,2020-01-01,1호선,서울역,승차,6,280,8725.0,2080.000000,중구,섬식,...,1,0,0,0,회현동,2020-01-01_회현동,2020-01-01_중구,-6.700000,1.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1867115,2020-06-28,6호선,연신내,하차,21,0,3241.0,2419.540039,은평구,상대식,...,2,0,1,1,불광제2동,2020-06-28_불광제2동,2020-06-28_은평구,22.200001,0.2,0.0
1867116,2020-06-28,6호선,연신내,승차,22,0,3241.0,2419.540039,은평구,상대식,...,2,0,1,1,불광제2동,2020-06-28_불광제2동,2020-06-28_은평구,20.900000,0.2,0.0
1867117,2020-06-28,6호선,연신내,하차,22,0,3241.0,2419.540039,은평구,상대식,...,2,0,1,1,불광제2동,2020-06-28_불광제2동,2020-06-28_은평구,20.900000,0.2,0.0
1867118,2020-06-28,6호선,연신내,승차,23,0,3241.0,2419.540039,은평구,상대식,...,2,0,1,1,불광제2동,2020-06-28_불광제2동,2020-06-28_은평구,20.299999,0.0,0.0


In [11]:
# subway_weather_final + pop_final
subway_weather_pop_final = pd.merge(subway_weather_final, pop_final, how = 'left', on = ['pop_join_key', 'date', 'hour'])
subway_weather_pop_final

,date,line,station_name,boarding,hour,boarding_num,waiting_area,platform_area,gu,type,...,windspeed,rain,total,pop_below_20,pop_20,pop_30,pop_40,pop_50,pop_60,pop_over_70
0,2020-01-01,1호선,서울역,승차,0,22,8725.0,2080.000000,중구,섬식,...,0.7,0.0,10730.0,900.0,1756.0,2088.0,2008.0,1627.0,1281.0,1073.0
1,2020-01-01,1호선,서울역,하차,0,74,8725.0,2080.000000,중구,섬식,...,0.7,0.0,10730.0,900.0,1756.0,2088.0,2008.0,1627.0,1281.0,1073.0
2,2020-01-01,1호선,서울역,승차,5,316,8725.0,2080.000000,중구,섬식,...,0.4,0.0,8999.0,715.0,1560.0,1799.0,1715.0,1329.0,999.0,884.0
3,2020-01-01,1호선,서울역,하차,5,234,8725.0,2080.000000,중구,섬식,...,0.4,0.0,8999.0,715.0,1560.0,1799.0,1715.0,1329.0,999.0,884.0
4,2020-01-01,1호선,서울역,승차,6,280,8725.0,2080.000000,중구,섬식,...,1.5,0.0,10471.0,857.0,1787.0,1926.0,1889.0,1675.0,1335.0,1006.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1867115,2020-06-28,6호선,연신내,하차,21,0,3241.0,2419.540039,은평구,상대식,...,0.2,0.0,23641.0,3058.0,2994.0,3439.0,3656.0,3908.0,3134.0,3454.0
1867116,2020-06-28,6호선,연신내,승차,22,0,3241.0,2419.540039,은평구,상대식,...,0.2,0.0,24108.0,3129.0,3117.0,3577.0,3738.0,3937.0,3138.0,3474.0
1867117,2020-06-28,6호선,연신내,하차,22,0,3241.0,2419.540039,은평구,상대식,...,0.2,0.0,24108.0,3129.0,3117.0,3577.0,3738.0,3937.0,3138.0,3474.0
1867118,2020-06-28,6호선,연신내,승차,23,0,3241.0,2419.540039,은평구,상대식,...,0.0,0.0,27219.0,3520.0,3580.0,3984.0,4142.0,4576.0,3578.0,3843.0


In [12]:
subway_weather_pop_final.isnull().sum()

date                      0
line                      0
station_name              0
boarding                  0
hour                      0
boarding_num              0
waiting_area              0
platform_area             0
gu                        0
type                      0
length                    0
floor                     0
transfer            1172080
transfer_num              0
entrance                  0
gate                  18000
msi_cnt                   0
hsi_cnt                   0
uvi_cnt                   0
hpi_cnt                   0
mti_cnt                   0
dong                      0
pop_join_key              0
weather_join_key          0
temp                      0
windspeed                 0
rain                      0
total                     0
pop_below_20              0
pop_20                    0
pop_30                    0
pop_40                    0
pop_50                    0
pop_60                    0
pop_over_70               0
dtype: int64

- 위와 같이 존재하는 결측치들에 대해서는 데이터 탐색 단계에서 처리해주도록 하겠다.
    - 우선은 조인이 잘 이루어졌는지만 확인하겠다.

- 조인이 잘 이루어졌으므로, 조인키로 사용했던 weather_join_key와 pop_join_key 변수는 제거해주도록 하겠다.

In [13]:
subway_weather_pop_final.drop(['weather_join_key', 'pop_join_key'], axis = 1, inplace = True)

In [14]:
subway_weather_pop_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1867120 entries, 0 to 1867119
Data columns (total 33 columns):
 #   Column         Dtype   
---  ------         -----   
 0   date           category
 1   line           category
 2   station_name   category
 3   boarding       category
 4   hour           int32   
 5   boarding_num   int32   
 6   waiting_area   float32 
 7   platform_area  float32 
 8   gu             category
 9   type           category
 10  length         int32   
 11  floor          category
 12  transfer       category
 13  transfer_num   int32   
 14  entrance       int32   
 15  gate           category
 16  msi_cnt        int32   
 17  hsi_cnt        int32   
 18  uvi_cnt        int32   
 19  hpi_cnt        int32   
 20  mti_cnt        int32   
 21  dong           category
 22  temp           float32 
 23  windspeed      float32 
 24  rain           float32 
 25  total          float32 
 26  pop_below_20   float32 
 27  pop_20         float32 
 28  pop_30      

# 최종 생성된 데이터 셋을 csv 파일로 내보내기

In [15]:
subway_weather_pop_final.to_csv('{}/df_test.csv'.format(filepath), index = False)